In [6]:
sc

In [3]:
df = spark.read.load('citibike.csv', format='csv', header=True, inferSchema=False)

In [4]:
df.dtypes[:5]

[('cartodb_id', 'string'),
 ('the_geom', 'string'),
 ('tripduration', 'string'),
 ('starttime', 'string'),
 ('stoptime', 'string')]

In [5]:
from pyspark.sql import Row
import csv

def parseCSV(idx, part):
    if idx==0:
        part.next()
    for p in csv.reader(part):
        yield Row(tripduration=int(p[2]),
                 starttime=p[3],
                  start_station_time=p[6])
        
rows = sc.textFile('citibike.csv').mapPartitionsWithIndex(parseCSV)
df = sqlContext.createDataFrame(rows)

In [6]:
df.dtypes

[('start_station_time', 'string'),
 ('starttime', 'string'),
 ('tripduration', 'bigint')]

In [16]:
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import csv

def parseCSV(idx, part):
    if idx==0:
        part.next()
    for p in csv.reader(part):
        yield (int(p[2]), p[3], p[6])
        
rows = sc.textFile('citibike.csv').mapPartitionsWithIndex(parseCSV)
schema = StructType([StructField('tripduration', IntegerType()),
                    StructField('starttime', StringType()),
                    StructField('start_station_name', StringType())])

df = sqlContext.createDataFrame(rows, schema)
df.dtypes


[('tripduration', 'int'),
 ('starttime', 'string'),
 ('start_station_name', 'string')]

In [19]:
import pyspark.sql.functions as sf

df.agg(sf.approx_count_distinct(df.tripduration).alias('c')).show()

+----+
|   c|
+----+
|2537|
+----+



In [20]:
df.agg(sf.approx_count_distinct((df.tripduration/60).cast('int')).alias('c')).show()

+---+
|  c|
+---+
|169|
+---+



In [13]:
df.registerTempTable('citibike')

In [14]:
sqlContext.sql('select * from citibike order by tripduration limit 5').collect()

[Row(start_station_time=u'W 20 St & 8 Ave', starttime=u'2015-02-06 11:38:00+00', tripduration=60),
 Row(start_station_time=u'Catherine St & Monroe St', starttime=u'2015-02-05 09:32:00+00', tripduration=60),
 Row(start_station_time=u'E 10 St & Avenue A', starttime=u'2015-02-06 20:40:00+00', tripduration=60),
 Row(start_station_time=u'E 5 St & Avenue C', starttime=u'2015-02-07 21:40:00+00', tripduration=60),
 Row(start_station_time=u'W 31 St & 7 Ave', starttime=u'2015-02-02 21:39:00+00', tripduration=60)]

In [15]:
sqlContext.sql('select percentile (tripduration, 0.5) from citibike').collect()

[Row(percentile(tripduration, CAST(0.5 AS DOUBLE))=529.0)]

In [4]:
SAT_FN = 'SAT_Results.csv'
HSD_FN = 'DOE_High_School_Directory_2014-2015.csv'

In [8]:
dfSchools = spark.read.load(HSD_FN, format='csv', header=True, inferSchema=True)\
                .na.drop(subset=['boro']) # Skipping multiple line recors
    
dfSchools = dfSchools.filter(dfSchools['total_students']>500)
dfSchools = dfSchools.select('dbn', 'boro')
dfSchools.count()

176

In [10]:
dfScores = sqlContext.read.load(SAT_FN, format='csv', header=True, inferSchema=True)#header is true bc file has header
dfScores.select(dfScores['`SAT Math Avg. Score`'].cast('int')) # ` is for quoting the name

DataFrame[SAT Math Avg. Score: int]

In [18]:
dfMScores = dfScores.select('DBN', dfScores['`SAT Math Avg. Score`'].cast('int').alias('score'), 
                            dfScores['Num of SAT Test Takers'].cast('int').alias('ntakers'),).na.drop()

# Some of the rows doesnt have an integer it has an s then it will drop that has null values 
# that is unable to cast to int

#dfMScores.show()
# +------+------+-------+
# |   DBN|scores|ntakers|
# +------+------+-------+
# |02M047|   400|    400|
# |21K410|   437|    437|
# |30Q301|   440|    440|
# |17K382|   374|    374|
# |18K637|   381|    381|
# |32K403|   366|    366|
# |09X365|   339|    339|
# |11X270|   394|    394|
# |05M367|   366|    366|
# |14K404|   357|    357|
# |30Q575|   492|    492|
# |13K336|   374|    374|
# |04M635|   369|    369|
# |24Q264|   454|    454|
# |17K408|   342|    342|
# |19K618|   371|    371|
# |27Q309|   379|    379|
# |32K552|   364|    364|
# |13K499|   364|    364|
# |07X600|   400|    400|
# +------+------+-------+
#only showing top 20 rows

dfMScores = dfMScores.select('DBN', (dfMScores.score*dfMScores.ntakers).alias('sum_scores'), 'ntakers')
dfMScores.show()

+------+----------+-------+
|   DBN|sum_scores|ntakers|
+------+----------+-------+
|02M047|      6400|     16|
|21K410|    207575|    475|
|30Q301|     43120|     98|
|17K382|     22066|     59|
|18K637|     13335|     35|
|32K403|     18300|     50|
|09X365|     18306|     54|
|11X270|     22064|     56|
|05M367|     12078|     33|
|14K404|     24276|     68|
|30Q575|     66420|    135|
|13K336|      3366|      9|
|04M635|     17712|     48|
|24Q264|     40406|     89|
|17K408|     19494|     57|
|19K618|     22260|     60|
|27Q309|     13644|     36|
|32K552|     24388|     67|
|13K499|     26208|     72|
|07X600|     30400|     76|
+------+----------+-------+
only showing top 20 rows



In [20]:




df = dfSchools.join(dfMScores, dfSchools.dbn==dfScores.DBN, how='inner')
#df.show()
# +------+---------+------+----------+-------+
# |   dbn|     boro|   DBN|sum_scores|ntakers|
# +------+---------+------+----------+-------+
# |01M450|Manhattan|01M450|     28140|     70|
# |01M539|Manhattan|01M539|     91266|    159|
# |01M696|Manhattan|01M696|     78520|    130|
# |02M374|Manhattan|02M374|     23460|     60|
# |02M400|Manhattan|02M400|    129659|    263|
# |02M408|Manhattan|02M408|     46060|     94|
# |02M412|Manhattan|02M412|     67260|    114|
# |02M413|Manhattan|02M413|     35178|     66|
# |02M416|Manhattan|02M416|     75438|    127|
# |02M418|Manhattan|02M418|     79632|    144|
# |02M420|Manhattan|02M420|    150864|    336|
# |02M425|Manhattan|02M425|     35784|     84|
# |02M475|Manhattan|02M475|    611520|    832|
# |02M489|Manhattan|02M489|     81663|    167|
# |02M519|Manhattan|02M519|     38232|     81|
# |02M520|Manhattan|02M520|    116160|    264|
# |02M529|Manhattan|02M529|     55675|    131|
# |02M542|Manhattan|02M542|     24948|     66|
# |02M580|Manhattan|02M580|     32256|     84|
# |02M600|Manhattan|02M600|    141705|    335|
# +------+---------+------+----------+-------+
# only showing top 20 rows

df = df.groupBy('boro').sum('sum_scores','ntakers') 
df.show()

+-------------+---------------+------------+
|         boro|sum(sum_scores)|sum(ntakers)|
+-------------+---------------+------------+
|       Queens|        5190534|       10942|
|     Brooklyn|        4544126|        9322|
|Staten Island|        1406967|        2944|
|    Manhattan|        3206992|        6228|
|        Bronx|        1619364|        3444|
+-------------+---------------+------------+



In [21]:
df.withColumn('avg', df[1]/df[2]).show()

+-------------+---------------+------------+------------------+
|         boro|sum(sum_scores)|sum(ntakers)|               avg|
+-------------+---------------+------------+------------------+
|       Queens|        5190534|       10942| 474.3679400475233|
|     Brooklyn|        4544126|        9322|487.46256168204246|
|Staten Island|        1406967|        2944| 477.9099864130435|
|    Manhattan|        3206992|        6228| 514.9312780989081|
|        Bronx|        1619364|        3444|  470.198606271777|
+-------------+---------------+------------+------------------+



In [22]:
df.withColumn('avg', (df[1]/df[2]).cast('int')).select('boro', 'avg').show()

+-------------+---+
|         boro|avg|
+-------------+---+
|       Queens|474|
|     Brooklyn|487|
|Staten Island|477|
|    Manhattan|514|
|        Bronx|470|
+-------------+---+



## TWITTER STUFF

In [23]:
dfTweets = sqlContext.read.load('twitter_1k.jsonl', format='json')
dfTweets.printSchema() # assumes that has a right structure

# TASK: Getting all the hashtags count

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true

In [30]:
import pyspark.sql.functions as sf

dfTweets = sqlContext.read.load('twitter_1k.jsonl', format='json')
hashtags = dfTweets.select('entities.hashtags.text')
hashtags = hashtags.filter(sf.size('text')>0) #removes the empty fields text > 0
#hashtags.show() 
# +--------------------+
# |                text|
# +--------------------+
# | [privatepassSports]|
# |[MaddenNeverLeave...|
# |[rave, edm, plur,...|
# |             [Bulls]|
# |[czechgirls, czec...|
# |[USSaluteouttuesday]|
# | [Scan961, Viviendo]|
# |[USSaluteouttuesday]|
# |           [fly, ho]|
# |[USSaluteouttuesday]|
# |        [Sacrifices]|
# |[USSaluteouttuesday]|
# |          [porqueno]|
# |[takemebacktoBarc...|
# |[USSaluteouttuesday]|
# |[if, ifsoho, soho...|
# |[USSaluteouttuesday]|
# |[dayoff, relaxday...|
# |            [Dexter]|
# |[USSaluteouttuesday]|
# +--------------------+
#only showing top 20 rows
hashtags = hashtags.select(sf.explode('text').alias('hashtag')) # list of list and make into a single list 
hashtags = hashtags.groupBy('hashtag').count().orderBy('count', ascending=False)
hashtags.show()

+------------------+-----+
|           hashtag|count|
+------------------+-----+
|USSaluteouttuesday|   33|
|               NYC|    5|
| privatepassSports|    4|
|               nyc|    4|
|         superbowl|    3|
|       RobinThicke|    3|
|              espn|    3|
|               Job|    3|
|   SuperBowlXLVIII|    3|
|         redcarpet|    3|
|      ESPNTHEPARTY|    2|
|             ncaaw|    2|
|              Jobs|    2|
|             facts|    2|
|               Raw|    2|
|           NewYork|    2|
|              soho|    2|
|    henleyvaporium|    2|
|                IT|    2|
|         SuperBowl|    2|
+------------------+-----+
only showing top 20 rows



In [ ]:
# WHEN RUNNING YOUR SPARK SCRIPT ON THE CLUSTER 

# sc = SparkContext()
# sqlContext = SQLContext(sc)

# checking error, you need to do yarn error -applicationID <application id>